In [ ]:
# First time setup - run this only once to save cookies
import pickle
from selenium import webdriver

def first_time_setup(url_login):
    try:
        driver = webdriver.Chrome()
        driver.get(url_login)
        
        print(f"Opening {url_login}")
        print("Please login manually in the browser window...")
        print("After logging in successfully, press Enter in this console...")
        input()  # Wait for user to login and press Enter
        
        # After manual login, save the cookies
        cookies = driver.get_cookies()
        with open('cookies.pkl', 'wb') as file:
            pickle.dump(cookies, file)
        print("Cookies saved successfully!")
        return driver
    except Exception as e:
        print(f"Error: {e}")
# Usage:
url_login = "https://authn.edx.org/login"
driver = first_time_setup(url_login)

In [ ]:
# import pickle
import pickle
# import webdriver
from selenium import webdriver
from bs4 import BeautifulSoup

# For subsequent runs - use this function
# return soup variable for further processing
def load_login(url_login):
    try:
        driver = webdriver.Chrome()
        driver.get(url_login)  # Go to login page first
        
        # Load saved cookies
        with open('cookies.pkl', 'rb') as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
        
        return driver
    except Exception as e:
        print(f"Error: {e}")
        
url_login = "https://authn.edx.org/login"
driver = load_login(url_login)

# create soup object()
url_scrape = "https://learning.edx.org/course/course-v1:HarvardX+MCB63X+1T2024c/block-v1:HarvardX+MCB63X+1T2024c+type@sequential+block@b302fa451b5b46d9957b0a26ab08443b/block-v1:HarvardX+MCB63X+1T2024c+type@vertical+block@9fc58c08f3a649ec954b4ce2de67c3d3"
driver.get(url_scrape)
# Wait for a specific element to load
try:
    WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.ID, "root")))  # Change "root" to a real element if needed
except:
    print("Timeout waiting for page to load")

# Get updated page source
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup.prettify())



In [18]:
# create soup object()
url_scrape = "http://udemy.com/course/biostatistics-fundamentals-using-python/learn/lecture/10693312?start=0#overview"
driver.get(url_scrape)
# Wait for a specific element to load
try:
    WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.ID, "root")))  # Change "root" to a real element if needed
except:
    print("Timeout waiting for page to load")

# Get updated page source
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup.prettify())

/Users/olivero54/Dropbox/z-dev/dvw-scrape-biochemcourse/.venv/lib/python3.12/site-packages/pygments/regexopt.py:77: RuntimeWarning: coroutine 'scrape_with_playwright' was never awaited
  '|'.join(regex_opt_inner(list(group[1]), '')


MaxRetryError: HTTPConnectionPool(host='localhost', port=60225): Max retries exceeded with url: /session/9877df661f99dd22248be48609f79b65/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1148567e0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [17]:
driver.quit()

In [ ]:
import asyncio
import pickle
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

COOKIE_FILE = "cookies.pkl"
URL_LOGIN = "https://authn.edx.org/login"
URL_SCRAPE = "https://learning.edx.org/course/course-v1:HarvardX+MCB63X+1T2024c/block-v1:HarvardX+MCB63X+1T2024c+type@sequential+block@b302fa451b5b46d9957b0a26ab08443b/block-v1:HarvardX+MCB63X+1T2024c+type@vertical+block@9fc58c08f3a649ec954b4ce2de67c3d3"

async def save_cookies(context):
    """Save cookies to a file."""
    cookies = await context.storage_state()
    with open(COOKIE_FILE, "wb") as file:
        pickle.dump(cookies, file)

async def load_cookies(context):
    """Load cookies from a file if available."""
    try:
        with open(COOKIE_FILE, "rb") as file:
            cookies = pickle.load(file)
            await context.add_cookies(cookies["cookies"])
    except FileNotFoundError:
        print("No saved cookies found. Logging in may be required.")

async def scrape_with_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Change to True to run headless
        context = await browser.new_context()
        page = await context.new_page()

        # Load cookies if available
        await load_cookies(context)

        # Go to the login page and allow manual login if needed
        await page.goto(URL_LOGIN)
        await asyncio.sleep(5)  # Give time for manual login if needed

        # Save cookies after logging in
        await save_cookies(context)

        # Navigate to the target page
        await page.goto(URL_SCRAPE)

        # Wait for content to load (adjust selector as needed)
        await page.wait_for_selector("#root", timeout=10000)

        # Get page content
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        print(soup.prettify())

        await browser.close()

# Run the Playwright scraping function
asyncio.run(scrape_with_playwright())
